In [3]:
!pip freeze

absl-py==1.0.0
aiohttp==3.8.1
aiosignal==1.2.0
anyio==3.6.1
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
asttokens==2.0.5
astunparse==1.6.3
async-timeout==4.0.1
attrs==21.2.0
Babel==2.10.3
backcall==0.2.0
beautifulsoup4==4.6.0
black==22.6.0
bleach==5.0.1
cachetools==4.2.4
certifi==2021.10.8
cffi==1.15.1
charset-normalizer==2.0.7
click==8.0.3
coloredlogs==15.0.1
cssselect==1.1.0
cycler==0.11.0
datasets==1.13.3
debugpy==1.6.2
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.4
entrypoints==0.4
executing==0.9.1
fastjsonschema==2.16.1
feedfinder2==0.0.4
feedparser==6.0.10
filelock==3.4.0
Flask==2.0.2
Flask-API==3.0.post1
flatbuffers==1.12
fonttools==4.34.4
frozenlist==1.2.0
fsspec==2021.11.0
future==0.18.2
gast==0.4.0
google-auth==2.3.3
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
greenlet==1.1.2
grpcio==1.41.1
h5py==3.7.0
huggingface-hub==0.8.1
humanfriendly==10.0
idna==3.3
importlib-metadata==4.8.2
ipykernel==6.15.1
ipython==8.4.0
ipython-genutils==0.2.0
itsdangerous==2.0.1
jedi==0.18.

In [1]:
import pandas as pd
import numpy as np
import os
# from konlpy import Mecab

!pwd

/root/2022_chatbot/assets/2022Military-Chatbot-jupyter/source


In [ ]:
WORK_DIR="/root/2022_chatbot/assets"
MECAB_DIR="/root/2022_chatbot/mecab-ko-dic-2.1.1-20180720"

%cd $WORK_DIR
!ls

/root/2022_chatbot/assets
2022Military-Chatbot-jupyter  tmp	      user_nnp_backup.csv
nnp.csv			      Untitled.ipynb  _user-nnp.csv


In [9]:
# 미리 만들어둔 사용자 사전 nnp파일 read
with open(WORK_DIR+"/nnp.csv", 'r', encoding='utf-8') as f:
  file_new = f.readlines()
file_new

['대우,,,,NNP,*,F,대우,*,*,*,*,*\n',
 '구글,,,,NNP,*,T,구글,*,*,*,*,*\n',
 '모바일서비스팀,,,,NNP,*,F,모바일서비스팀,*,*,*,*,*']

In [72]:
from konlpy.tag import Mecab
mecab = Mecab()

In [33]:
mecab.morphs("살아남는다는 것에 대하여")

['살아남', '는다는', '것', '에', '대하', '여']

In [40]:
mecab.morphs("양주옥정신도시 3차 노블랜드 에듀포레")

['양', '주옥', '정신', '도시', '3', '차', '노', '블랜드', '에듀', '포레']

In [54]:
ord("나")

45208

## Sample - 계급 추가

In [57]:
def is_hangul(word):
    for w in word:
        if (44032 <= ord(w) and ord(w) <= 55203) or w == ' ' or (ord(w) >= 48 and ord(w) <= 57) or (ord(w) >= 65 and ord(w) <= 122):
            pass
        else:
            return False
    
    return True

def has_coda(word):
    return (ord(word[-1])-44032) %28 == 0

# 종성유무 반환 함수
has_coda('대령')
is_hangul('메 롱')
is_hangul('메 롱3')

True

In [58]:
sample_df = pd.read_csv(WORK_DIR+'/tmp/계급.csv')
sample_df

,0
0,소장
1,준장
2,대령
3,중령
4,소령
5,대위
6,중위
7,소위
8,준위
9,원사


In [15]:
with open(WORK_DIR+"/nnp.csv", 'w', encoding='utf-8') as f:
    for w in sample_df['0']:
        coda = 'F' if has_coda(w) else 'T'
        line = f'{w},,,,NNP,*,{coda},{w},*,*,*,*\n'
        f.write(line)

%cat nnp.csv

소장,,,,NNP,*,T,소장,*,*,*,*
준장,,,,NNP,*,T,준장,*,*,*,*
대령,,,,NNP,*,T,대령,*,*,*,*
중령,,,,NNP,*,T,중령,*,*,*,*
소령,,,,NNP,*,T,소령,*,*,*,*
대위,,,,NNP,*,F,대위,*,*,*,*
중위,,,,NNP,*,F,중위,*,*,*,*
소위,,,,NNP,*,F,소위,*,*,*,*
준위,,,,NNP,*,F,준위,*,*,*,*
원사,,,,NNP,*,F,원사,*,*,*,*
상사,,,,NNP,*,F,상사,*,*,*,*
중사,,,,NNP,*,F,중사,*,*,*,*
하사,,,,NNP,*,F,하사,*,*,*,*
병장,,,,NNP,*,T,병장,*,*,*,*
상병,,,,NNP,*,T,상병,*,*,*,*
일병,,,,NNP,*,T,일병,*,*,*,*
이병,,,,NNP,*,T,이병,*,*,*,*


## nnp 단어추가

In [59]:
with open(WORK_DIR+"/nnp.csv", 'w', encoding='utf-8') as f:
    for dirname, _, filenames in os.walk(WORK_DIR+'/tmp'):
        for filename in filenames:
            filepath = os.path.join(dirname, filename)
            sample_df = pd.read_csv(filepath)
            
            fnm = filename.split(".")[0]
            print(f'[{fnm}] # of data : {len(sample_df.index)} ===================>')
            tag = 'NNP'
            nm = '*'
            
            if fnm=='저자':
                nm = '인명'
            elif fnm=='지역':
                nm = '지명'
            
            for w in sample_df['0']:
                try:
                    if is_hangul(w) == True:
                        coda = 'F' if has_coda(w) else 'T'
                        line = f'{w},,,,NNP,{nm},{coda},{w},*,*,*,*,*\n'
                        f.write(line)  
                except:
                    continue

[저자_cp949] # of data : 68047 ===================>
[군구분] # of data : 4 ===================>
[지역] # of data : 284 ===================>
[시설] # of data : 188 ===================>
[주택] # of data : 2093 ===================>
[부대번호] # of data : 169 ===================>
[부대] # of data : 18071 ===================>
[도서명_cp949] # of data : 118643 ===================>
[직급] # of data : 23 ===================>
[계급] # of data : 17 ===================>
[식단] # of data : 18 ===================>


## 우선순위 단어 조정
#### 사전추가 전 테스트

In [10]:
tests = pd.read_csv("/root/2022_chatbot/rest-server/tests/assets/x_test.csv")

In [14]:
from konlpy.tag import Mecab
mecab = Mecab()

# for i in range(len(tests)):
for i in range(100,200):
    ctx = tests['question'][i]
    print(f"{ctx} => {mecab.morphs(ctx)}")

자기 전에 통화하면 썸이야? => ['자기', '전', '에', '통화', '하', '면', '썸', '이', '야', '?']
가평군부근 군 관사 있을까? => ['가평군', '부근', '군', '관사', '있', '을까', '?']
상사 7호봉 연봉 어느정도임 => ['상사', '7', '호봉', '연봉', '어느', '정도', '임']
하사 33호봉 봉급 궁금함 => ['하사', '33', '호봉', '봉급', '궁금', '함']
서비스센터 가야하게 생겼네 => ['서비스', '센터', '가', '야하', '게', '생겼', '네']
가격 쓰여져 있는 전단지 같은거 있으면 주시겠어요? => ['가격', '쓰여져', '있', '는', '전단지', '같', '은', '거', '있', '으면', '주', '시', '겠', '어요', '?']
사랑하는데 꼭 결혼해야할지는 모르겠어요 => ['사랑', '하', '는데', '꼭', '결혼', '해야', '할지', '는', '모르', '겠', '어요']
담배끊었었는데 담배 피고 싶다 => ['담배', '끊', '었었', '는데', '담배', '피', '고', '싶', '다']
자취방 있나요? => ['자취방', '있', '나요', '?']
로스먼, 요한나 이 쓴 간행물 있을까 => ['로스먼', ',', '요한나', '이', '쓴', '간행물', '있', '을까']
Clancy,Tom 이 저자인 도서 정기간행물이름 => ['Clancy', ',', 'Tom', '이', '저자', '인', '도서', '정기', '간행물', '이름']
장학금 가능할까? => ['장학금', '가능', '할까', '?']
백산주택에 대해 알고싶어 => ['백산', '주택', '에', '대해', '알', '고', '싶', '어']
카드로 계산해도 괜찮나요? => ['카드', '로', '계산', '해도', '괜찮', '나요', '?']
군대 기다릴 수 있을까 => ['군대', '기다릴', '수', '있', '을까']
오송지역 복지정보 있을까? 

#### nnp => user-nnp 변환

In [22]:
%cd $MECAB_DIR

/root/2022_chatbot/mecab-ko-dic-2.1.1-20180720


In [35]:
with open("user-person.csv", 'r', encoding='utf-8') as f:
    userdic = f.readlines()

In [36]:
userdic[:10]

['Academy of Management,1786,3546,2953,NNP,*,T,Academy of Management,*,*,*,*,*\n',
 'Sage Publications,1786,3546,2953,NNP,*,T,Sage Publications,*,*,*,*,*\n',
 'Air Force Research Institute,1786,3546,2953,NNP,*,T,Air Force Research Institute,*,*,*,*,*\n',
 'Key Publishing Ltd,1786,3545,2953,NNP,*,F,Key Publishing Ltd,*,*,*,*,*\n',
 'American Economic Association,1786,3546,2953,NNP,*,T,American Economic Association,*,*,*,*,*\n',
 'Cambridge University Press,1786,3546,2953,NNP,*,T,Cambridge University Press,*,*,*,*,*\n',
 'University of Chicago Press,1786,3546,2953,NNP,*,T,University of Chicago Press,*,*,*,*,*\n',
 'University of California Press,1786,3546,2953,NNP,*,T,University of California Press,*,*,*,*,*\n',
 'Comparative Politics,1786,3546,2953,NNP,*,T,Comparative Politics,*,*,*,*,*\n',
 'Current History,1786,3546,2953,NNP,*,T,Current History,*,*,*,*,*\n']

In [37]:
sample_line = userdic[0]
arr = sample_line.split(',')

In [38]:
arr[3] = '0'
ret = ','.join(arr)
ret

'Academy of Management,1786,3546,0,NNP,*,T,Academy of Management,*,*,*,*,*\n'

In [39]:
with open("user-person.csv", 'w', encoding='utf-8') as f:
    for line in userdic:
        arr = line.split(',')    
        arr[3] = '0'
        f.write(','.join(arr))

In [40]:
with open("user-person.csv", 'r', encoding='utf-8') as f:
    userdic2 = f.readlines()
userdic2[:10]

['Academy of Management,1786,3546,0,NNP,*,T,Academy of Management,*,*,*,*,*\n',
 'Sage Publications,1786,3546,0,NNP,*,T,Sage Publications,*,*,*,*,*\n',
 'Air Force Research Institute,1786,3546,0,NNP,*,T,Air Force Research Institute,*,*,*,*,*\n',
 'Key Publishing Ltd,1786,3545,0,NNP,*,F,Key Publishing Ltd,*,*,*,*,*\n',
 'American Economic Association,1786,3546,0,NNP,*,T,American Economic Association,*,*,*,*,*\n',
 'Cambridge University Press,1786,3546,0,NNP,*,T,Cambridge University Press,*,*,*,*,*\n',
 'University of Chicago Press,1786,3546,0,NNP,*,T,University of Chicago Press,*,*,*,*,*\n',
 'University of California Press,1786,3546,0,NNP,*,T,University of California Press,*,*,*,*,*\n',
 'Comparative Politics,1786,3546,0,NNP,*,T,Comparative Politics,*,*,*,*,*\n',
 'Current History,1786,3546,0,NNP,*,T,Current History,*,*,*,*,*\n']

#### 사용자 사전 추가 후

In [42]:
from konlpy.tag import Mecab
mecab = Mecab()

# for i in range(len(tests)):
for i in range(100,200):
    ctx = tests['question'][i]
    print(f"{ctx} => {mecab.morphs(ctx)}")

자기 전에 통화하면 썸이야? => ['자기', '전', '에', '통화', '하', '면', '썸', '이', '야', '?']
가평군부근 군 관사 있을까? => ['가평군', '부근', '군', '관사', '있', '을까', '?']
상사 7호봉 연봉 어느정도임 => ['상사', '7', '호봉', '연봉', '어느', '정도', '임']
하사 33호봉 봉급 궁금함 => ['하사', '33', '호봉', '봉급', '궁금', '함']
서비스센터 가야하게 생겼네 => ['서비스', '센터', '가', '야하', '게', '생겼', '네']
가격 쓰여져 있는 전단지 같은거 있으면 주시겠어요? => ['가격', '쓰여져', '있', '는', '전단지', '같', '은', '거', '있', '으면', '주', '시', '겠', '어요', '?']
사랑하는데 꼭 결혼해야할지는 모르겠어요 => ['사랑', '하', '는데', '꼭', '결혼', '해야', '할지', '는', '모르', '겠', '어요']
담배끊었었는데 담배 피고 싶다 => ['담배', '끊', '었었', '는데', '담배', '피', '고', '싶', '다']
자취방 있나요? => ['자취방', '있', '나요', '?']
로스먼, 요한나 이 쓴 간행물 있을까 => ['로스먼', ',', '요한나', '이', '쓴', '간행물', '있', '을까']
Clancy,Tom 이 저자인 도서 정기간행물이름 => ['Clancy', ',', 'Tom', '이', '저자', '인', '도서', '정기', '간행물', '이름']
장학금 가능할까? => ['장학금', '가능', '할까', '?']
백산주택에 대해 알고싶어 => ['백산주택', '에', '대해', '알', '고', '싶', '어']
카드로 계산해도 괜찮나요? => ['카드', '로', '계산', '해도', '괜찮', '나요', '?']
군대 기다릴 수 있을까 => ['군대', '기다릴', '수', '있', '을까']
오송지역 복지정보 있을까? => [